# Mavlink Camera
> Mavlink Camera Component for sending commands to a camera on a companion computer or GCS
> The server is on the companion computer and the client is on the ground station PC.

In [ ]:
#| default_exp mavlink.camera

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
import time, os, sys

from UAV.logging import logging
from UAV.mavlink.mavcom import MAVCom, time_since_boot_ms, time_UTC_usec, boot_time_str, date_time_str
from UAV.mavlink.component import Component, mavutil, mavlink, MAVLink
import threading
import cv2
import numpy as np

# from UAV.imports import *   # TODO why is this relative import on nbdev_export?


In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *
from fastcore.test import *

### Implementation of these commands:
>
> [MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS = 527](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS)
[MAV_CMD_REQUEST_CAMERA_INFORMATION = 523](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION)
[MAV_CMD_REQUEST_CAMERA_SETTINGS = 524](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS)
[MAV_CMD_REQUEST_STORAGE_INFORMATION = 525](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION)
[MAV_CMD_STORAGE_FORMAT = 526](https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT)
[MAV_CMD_SET_CAMERA_ZOOM = 531](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM)
[MAV_CMD_SET_CAMERA_FOCUS = 532](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_FOCUS)
[MAV_CMD_IMAGE_START_CAPTURE = 2000](https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE)
[MAV_CMD_IMAGE_STOP_CAPTURE = 2001](https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE)
> 
> [MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION = 2504](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION)
[MAV_CMD_REQUEST_VIDEO_STREAM_STATUS = 2505](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_STATUS)
[MAV_CMD_VIDEO_START_CAPTURE = 2500](https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE)
[MAV_CMD_VIDEO_STOP_CAPTURE = 2501](https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE)
[MAV_CMD_SET_CAMERA_MODE = 530](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE)
> 
**Note**
The simulated camera is implemented in PX4 [gazebo_camera_manager_plugin.cpp](https://github.com/PX4/PX4-SITL_gazebo-classic/blob/main/src/gazebo_camera_manager_plugin.cpp).


In [ ]:
#| export
# from pymavlink.dialects.v20 import ardupilotmega as mav
# from pymavlink.dialects.v20.ardupilotmega import MAVLink


NAN = float("nan")

"""
MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS = 527 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS
MAV_CMD_REQUEST_CAMERA_INFORMATION = 521 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
MAV_CMD_REQUEST_CAMERA_SETTINGS = 522 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS
MAV_CMD_REQUEST_STORAGE_INFORMATION = 525 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION
MAV_CMD_STORAGE_FORMAT = 526 # https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT
MAV_CMD_SET_CAMERA_ZOOM = 531 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
MAV_CMD_SET_CAMERA_FOCUS = 532 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_FOCUS
MAV_CMD_IMAGE_START_CAPTURE = 2000  # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
MAV_CMD_IMAGE_STOP_CAPTURE = 2001  # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE
MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION = 2504 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION
MAV_CMD_REQUEST_VIDEO_STREAM_STATUS = 2505 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_STATUS
MAV_CMD_VIDEO_START_CAPTURE = 2500 # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE
MAV_CMD_VIDEO_STOP_CAPTURE = 2501 # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE
MAV_CMD_SET_CAMERA_MODE = 530 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE

"""
CAMERA_INFORMATION = mavlink.MAVLINK_MSG_ID_CAMERA_INFORMATION # https://mavlink.io/en/messages/common.html#CAMERA_INFORMATION
CAMERA_SETTINGS = mavlink.MAVLINK_MSG_ID_CAMERA_SETTINGS # https://mavlink.io/en/messages/common.html#CAMERA_SETTINGS
STORAGE_INFORMATION = mavlink.MAVLINK_MSG_ID_STORAGE_INFORMATION # https://mavlink.io/en/messages/common.html#STORAGE_INFORMATION
CAMERA_CAPTURE_STATUS = mavlink.MAVLINK_MSG_ID_CAMERA_CAPTURE_STATUS # https://mavlink.io/en/messages/common.html#CAMERA_CAPTURE_STATUS
CAMERA_IMAGE_CAPTURED = mavlink.MAVLINK_MSG_ID_CAMERA_IMAGE_CAPTURED # https://mavlink.io/en/messages/common.html#CAMERA_IMAGE_CAPTURED


In [ ]:
#| export


class WaitMessage:
    """Wait for a specific message from the server"""
    def __init__(self, target_system, target_component):
        # self.target_system = None
        # self.target_component = None
        self._condition = None
        self._event = threading.Event()
        self._object = None
        self._log = logging.getLogger("uav.{}".format(self.__class__.__name__))
        # self._msg_id = None

    @property
    def log(self) -> logging.Logger:
        return self._log

    def set_condition(self, msg_id, target_system, target_component):
        """Set the condition function to generate event for a specific message received from the server"""
        assert target_system is not None and target_component is not None, "call set_target(target_system, target_component) first"
        # self._msg_id = msg_id
        # self.target_system = target_system
        # self.target_component = target_component
        self._condition = (lambda msg: msg.get_msgId() == msg_id
                                   and msg.get_srcSystem() == target_system
                                   and msg.get_srcComponent() == target_component)
        self._event.clear()

    def on_condition(self, msg):
        """Event for a specific received from the server"""
        if self._condition is not None:
            # self.log.debug(f"!!!! on_condition : {msg.get_msgId() = } {self._msg_id = } ")
            # self.log.debug(f"!{msg.get_msgId() == self._msg_id} and {msg.get_srcSystem() = } {self.target_system} and {msg.get_srcComponent() == self.target_component } ")
            # self.log.debug(f"!!!!  {self._condition(msg) = } ")
            if self._condition(msg):
                # self.log.debug(f"!!!! on_condition meet : {msg = } ")
                self._object = msg
                self._event.set()
        # except:
        #     pass


    def set(self, value):
        """Set the object and set the event."""
        self._object = value
        self._event.set()

    def clear(self):
        """Clear the object and clear the event."""
        self._object = None
        self._event.clear()

    def get(self, timeout=1):
        """Get the object if the event is set or wait until it's set with an optional timeout.

        Returns:
            The object if the event is set, or None if it times out or the event isn't set.
        """
        is_set = self._event.wait(timeout)
        if is_set:
            return self._object
        return None

    def is_set(self):
        """Check if the event is set."""
        return self._event.is_set()


In [ ]:
show_doc(WaitMessage)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#L57){target="_blank" style="float:right; font-size:smaller"}

### WaitMessage

>      WaitMessage (target_system, target_component)

Wait for a specific message from the server

In [ ]:
show_doc(WaitMessage.set_condition)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#L72){target="_blank" style="float:right; font-size:smaller"}

### WaitMessage.set_condition

>      WaitMessage.set_condition (msg_id, target_system, target_component)

Set the condition function to generate event for a specific message received from the server

In [ ]:
show_doc(WaitMessage.get)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#L107){target="_blank" style="float:right; font-size:smaller"}

### WaitMessage.get

>      WaitMessage.get (timeout=1)

Get the object if the event is set or wait until it's set with an optional timeout.

Returns:
    The object if the event is set, or None if it times out or the event isn't set.

In [ ]:
#| export

class CameraClient(Component):
    """Create a Viewsheen mavlink gimbal client component for send commands to a gimbal on a companion computer or GCS """

    def __init__(self,
                 source_component,  # used for component indication
                 mav_type,  # used for heartbeat MAV_TYPE indication
                 debug):  # logging level
        
        super().__init__(source_component=source_component, mav_type=mav_type, debug=debug)
        self.set_message_callback(self.on_message)
        self.wait_for_message = WaitMessage(self.target_system, self.target_component)

    def on_mav_connection(self):
        pass

    def on_message(self, msg):
        """Callback for a command received from the server"""
        print(f"CAMERA_Client  {msg} ")
        self.wait_for_message.on_condition(msg)
        # if msg.get_type() == "CAMERA_IMAGE_CAPTURED":
        #     # print(f"Camera Capture Status {msg = }")
        #     print(msg)
        #         # f"{msg.image_count = }  {msg.image_interval = } {msg.recording_time_ms = } {msg.available_capacity = } {msg.image_status = } {msg.video_status = } {msg.image_interval = } ")
        #
        #     return True
            # print(f"Camera Image Captured {msg = }")


    def send_message(self, msg):
        """Send a message to the camera"""
        self.master.mav.send(msg)
        self.log.debug(f"Sent {msg}")

    def request_camera_capture_status(self):
        """Request camera capture status"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS
        # see https://github.com/PX4/PX4-SITL_gazebo-classic/blob/main/src/gazebo_camera_manager_plugin.cpp#L543

        assert self.target_system is not None and self.target_component is not None, "call set_target(target_system, target_component) first"
        self.wait_for_message.set_condition(CAMERA_CAPTURE_STATUS, self.target_system, self.target_component)
        t = self.send_command(  self.target_system,
                                self.target_component,
                                mavlink.MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS,
                                [1,0,0,0,0,0,0]
                              )
        return self.wait_for_message.get()


    def request_camera_information(self):
        """Request camera information"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
        assert self.target_system is not None and self.target_component is not None, "call set_target(target_system, target_component) first"
        self.wait_for_message.set_condition(CAMERA_INFORMATION, self.target_system, self.target_component)
        t = self.send_command(  self.target_system,
                                self.target_component,
                                mavlink.MAV_CMD_REQUEST_CAMERA_INFORMATION,
                                [1,0,0,0,0,0,0]
                              )
        return self.wait_for_message.get()

    def request_camera_settings(self):
        """Request camera settings"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS
        self.wait_for_message.set_condition(CAMERA_SETTINGS, self.target_system, self.target_component)
        t = self.send_command(  self.target_system,
                                self.target_component,
                                mavlink.MAV_CMD_REQUEST_CAMERA_SETTINGS,
                                [0,0,0,0,0,0,0]
                              )

        return self.wait_for_message.get()

    def request_storage_information(self):
        """Request storage information (for cases where camera has storage)"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION
        self.wait_for_message.set_condition(STORAGE_INFORMATION, self.target_system, self.target_component)
        t = self.send_command(  self.target_system,
                                self.target_component,
                                mavlink.MAV_CMD_REQUEST_STORAGE_INFORMATION,
                                [0,0,0,0,0,0,0]
                              )

        return self.wait_for_message.get()

    def storage_format(self):
        """Format storage (for cases where camera has storage)"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT
        self.wait_for_message.set_condition(STORAGE_INFORMATION, self.target_system, self.target_component)
        t = self.send_command(  self.target_system,
                                self.target_component,
                                mavlink.MAV_CMD_STORAGE_FORMAT,
                                [0,0,0,0,0,0,0]
                              )
        return self.wait_for_message.get()


    def set_camera_mode(self, mode_id): # https://mavlink.io/en/messages/common.html#CAMERA_MODE
        """ Set the camera mode"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE
        # https://mavlink.io/en/messages/common.html#CAMERA_MODE

        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_SET_CAMERA_MODE,
                                [0,
                                 mode_id, # https://mavlink.io/en/messages/common.html#CAMERA_MODE
                                 0,0,0,0,0]
                              )


    def set_camera_zoom(self, value): # 0 to 100 zoom value
        """ Set the camera zoom"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
        self.wait_for_message.set_condition(CAMERA_SETTINGS, self.target_system, self.target_component)
        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_SET_CAMERA_ZOOM,
                                [0,  # https://mavlink.io/en/messages/common.html#CAMERA_ZOOM_TYPE
                                 value, 0,0,0,0,0]
                              )

        return self.wait_for_message.get()


    def image_start_capture(self, interval=0, # Image capture interval
                            count=1, # Number of images to capture (0 for unlimited)
                            ):
        """Start image capture sequence."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
        try:
            self.image_capture_sequence_number += 1
            if count != 1:
                self.image_capture_sequence_number = 0
        except:
            self.image_capture_sequence_number = 1

        self.wait_for_message.set_condition(CAMERA_IMAGE_CAPTURED, self.target_system, self.target_component)
        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_IMAGE_START_CAPTURE,
                                [0,
                                 interval, # interval
                                 count, # number of  images to capture
                                 self.image_capture_sequence_number, # Sequence number starting from 1. This is only valid for single-capture (param3 == 1), otherwise set to 0.  Increment the capture ID for each capture command to prevent double captures when a command is re-transmitted.
                                 NAN, # Reserved
                                 NAN, # Reserved
                                 NAN]
                              ) # Reserved
        return self.wait_for_message.get()


    def image_stop_capture(self):
        """Stop image capture sequence"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE
        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_IMAGE_STOP_CAPTURE,
                                [0, NAN, NAN, NAN, NAN, NAN, NAN]
                              )

    def video_start_capture(self, video_stream_id=0, # Video stream id (0 for all streams)
                            frequency=0): # Frequency CAMERA_CAPTURE_STATUS messages should be sent while recording (0 for no messages, otherwise frequency in Hz)
        """Start video capture"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE
        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_VIDEO_START_CAPTURE,
                                [video_stream_id, # Video stream id (0 for all streams)
                                 frequency, # Frequency CAMERA_CAPTURE_STATUS messages should be sent while recording (0 for no messages, otherwise frequency in Hz)
                                 NAN, NAN, 0, 0, NAN]
                              )

    def video_stop_capture(self, video_stream_id=0): # Video stream id (0 for all streams)
        """Stop video capture"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE
        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_VIDEO_STOP_CAPTURE,
                                [video_stream_id, NAN, NAN, NAN, NAN, NAN, NAN]
                              )


In [ ]:
show_doc(CameraClient)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#L124){target="_blank" style="float:right; font-size:smaller"}

### CameraClient

>      CameraClient (mav_connection, source_component, mav_type, debug)

Create a Viewsheen mavlink gimbal client component for send commands to a gimbal on a companion computer or GCS

|    | **Details** |
| -- | ----------- |
| mav_connection | MavLinkBase connection |
| source_component | used for component indication |
| mav_type | used for heartbeat MAV_TYPE indication |
| debug | logging level |

In [ ]:
#| export


from fs.memoryfs import MemoryFS
class FakeCamera:
    """Create a fake camera component for testing"""
    def __init__(self, mav=None):
        self.mav:MAVLink = mav
        self.vendor_name = [i for i in range(32)]
        self.model_name = [i + 100 for i in range(32)]
        self.firmware_version = 0
        self.focal_length = 8.0
        self.sensor_size_h = 4.0
        self.sensor_size_v = 6.0
        self.resolution_h = 4000
        self.resolution_v = 3000
        self.lens_id = 10
        self.flags = 0
        self.cam_definition_version = 0
        self._cam_definition_uri_raw = b''
        self.cam_definition_uri = b''
        self.interval = 1
        self.max_count = 0
        self.current_img_cnt = 0
        self.image_index = 0
        self.image_filename = ""

        self.capture_thread = threading.Thread(target=self._capture_images)
        self.mem_fs = MemoryFS()
        self.fs_size = 100000000  # 100MB

    def save_image_to_memoryfs(self, img, filename):
        # Convert OpenCV image to JPEG byte stream
        success, buffer = cv2.imencode(".jpg", img)
        if not success:
            raise ValueError("Failed to encode image")

        # Write to PyFilesystem's Memory Filesystem
        with self.mem_fs.open(filename, "wb") as f:
            f.write(buffer.tobytes())

        print(f"Image saved to memory filesystem with name: {filename}")
        # return mem_fs

    def calculate_memory_usage(self):
        """Calculate total memory used by the MemoryFS."""
        total_memory = 0
        for path in self.mem_fs.walk.files():
            with self.mem_fs.open(path, "rb") as f:
                total_memory += len(f.read())
        return total_memory


    def camera_information_send(self):
        """ Information about a camera. Can be requested with a
            MAV_CMD_REQUEST_MESSAGE command."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
        self.mav.camera_information_send(time_since_boot_ms(),  # time_boot_ms
                                            self.vendor_name,   # vendor name
                                            self.model_name,    # model name
                                            self.firmware_version,    # firmware version
                                            self.focal_length,    # focal length
                                            self.sensor_size_h,   # sensor size h
                                            self.sensor_size_v,   # sensor size v
                                            self.resolution_h,    # resolution h
                                            self.resolution_v,    # resolution v
                                            self.lens_id,    # lend_id
                                            self.flags,     # flags
                                            self.cam_definition_version, # cam definition version
                                            self.cam_definition_uri,     # cam definition uri
                                         )

    def camera_settings_send(self):
        """ Information about a camera. Can be requested with a
            MAV_CMD_REQUEST_MESSAGE command."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS
        self.mav.camera_settings_send(time_since_boot_ms(),  # time_boot_ms
                                            0,   # camera_id
                                            0,    # settings_id
                                            0,    # value
                                         )

    def storage_information_send(self):
        """ Information about a camera. Can be requested with a
            MAV_CMD_REQUEST_MESSAGE command."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION
        self.mav.storage_information_send(time_since_boot_ms(),  # time_boot_ms
                                            0,   # storage_id
                                            1,    # storage_count
                                            0,    # status
                                            self.fs_size,    # total_capacity
                                            self.calculate_memory_usage(),    # used_capacity
                                            self.fs_size-self.calculate_memory_usage(),    # available_capacity
                                            0,    # read_speed
                                            0,    # write_speed
                                         )

    def camera_capture_status_send(self):
        """ Information about a camera. Can be requested with a
            MAV_CMD_REQUEST_MESSAGE command."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS
        self.mav.camera_capture_status_send(time_since_boot_ms(),  # time_boot_ms
                                            0,   # image_status
                                            0,    # video_status
                                            0,    # image_interval
                                            0,    # recording_time_ms
                                            0,    # available_capacity
                                            0,    # image_count
                                         )

    def camera_image_captured_send(self):
        if self.mav is not None:
            self.mav.camera_image_captured_send(time_since_boot_ms(),  # time_boot_ms
                                                time_UTC_usec(),  # time_utc
                                                0,  # camera_id
                                                0,  # lat
                                                0,  # lon
                                                0,  # alt
                                                0,  # relative_alt
                                                [0, 0, 0, 0],  # q
                                                self.image_index,  # image_index
                                                1,  # capture_result
                                                bytes(self.image_filename, 'utf-8'),  # file_url
                                                )

    def image_start_capture(self, interval, # Image capture interval
                            count, # Number of images to capture (0 for unlimited)
                            ):
        """Start image capture sequence."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
        self.interval = interval
        self.max_count = count
        self.capture_thread.start()

    def _capture_image(self, filename):
        # Simulate an image capture using OpenCV
        image = np.zeros((512, 512, 3), dtype=np.uint8)
        cv2.putText(image, "Fake Image", (50, 256), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        self.save_image_to_memoryfs(image, filename)

        # print(f"Saving image to {filename}")
        # is_success, buffer = cv2.imencode(".jpg", image)
        # with self.fs as my_fs:  # Replace './my_directory' with your desired directory
        #     with my_fs.open(filename, 'wb') as file:
        #         file.write(buffer.tobytes())
        # cv2.imwrite(filename, image)

    # 
    # def _capture_image2(self, save_path="image.jpg"):
    #     pipeline_str = "videotestsrc num-buffers=1 ! jpegenc ! filesink location={}".format(save_path)
    #     pipeline = Gst.parse_launch(pipeline_str)
    #     pipeline.set_state(Gst.State.PLAYING)
    #     bus = pipeline.get_bus()
    #     bus.timed_pop_filtered(Gst.CLOCK_TIME_NONE, Gst.MessageType.EOS | Gst.MessageType.ERROR)
    #     pipeline.set_state(Gst.State.NULL)
    def time_UTC_usec(self):
        return int(time.time() * 1e6)

    def _capture_images(self):
        self._capture_images_thread_running = True
        self.current_img_cnt = 0
        while self._capture_images_thread_running:
            self.image_filename = f"{date_time_str()}_{self.image_index:04d}.jpg"
            img = self._capture_image(self.image_filename)

            # print(f"Captured image")

            if self.mav is not None:
                self.mav.camera_image_captured_send(time_since_boot_ms(), # time_boot_ms
                                                    time_UTC_usec(), # time_utc
                                                    0, # camera_id
                                                    0, # lat
                                                    0, # lon
                                                    0, # alt
                                                    0, # relative_alt
                                                    [0,0,0,0], # q
                                                    self.image_index, # image_index
                                                    1, # capture_result
                                                    bytes(self.image_filename, 'utf-8'), # file_url
                                                    )
            time.sleep(self.interval)
            if self.current_img_cnt > self.max_count:
                self._capture_images_thread_running = False

            self.current_img_cnt += 1
            self.image_index += 1

    # def camera_capture_status_send(self, time_boot_ms, image_status, video_status, image_interval, recording_time_ms, available_capacity, image_count):
    #     msg = self.mav.mav.camera_capture_status_encode(time_boot_ms, image_status, video_status, image_interval, recording_time_ms, available_capacity, image_count)
    #     self.mav.mav.send(msg)



In [ ]:
show_doc(FakeCamera)

---

### FakeCamera

>      FakeCamera (mav=None)

Create a fake camera component for testing

In [ ]:
show_doc(FakeCamera.start_capture)

---

### FakeCamera.start_capture

>      FakeCamera.start_capture (interval, count)

Start image capture sequence.

|    | **Details** |
| -- | ----------- |
| interval | Image capture interval |
| count | Number of images to capture (0 for unlimited) |

In [ ]:
#| export

class CameraServer(Component):
    """Create a mavlink Camera server Component using a test GSTREAMER pipeline"""

    def __init__(self,
                 source_component,  # used for component indication
                 mav_type,  # used for heartbeat MAV_TYPE indication
                 camera, # camera  (or FakeCamera for testing)
                 debug,  # logging level
                ):
        super().__init__( source_component=source_component, mav_type=mav_type, debug=debug)
        self.set_message_callback(self.on_message)


        self.camera = camera


    def on_mav_connection(self):
        """Start the mavlink connection"""
        self.camera.mav = self.mav  # set the mavlink connection for mavlink messages


    def on_message(self, msg # type: Message
                   ) -> bool: # return True to indicate that the message has been handled
        """Callback for a command received from the client
        """

        if msg.get_type() == "COMMAND_LONG":
            # print(f"Command  {msg.command = } ")
            if msg.command == mavlink.MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS:
                self.camera_capture_status_send()
                return True # return True to indicate that the message has been handled

            elif msg.command == mavlink.MAV_CMD_REQUEST_CAMERA_INFORMATION:
                self.camera_information_send()
                return True # return True to indicate that the message has been handled

            elif msg.command == mavlink.MAV_CMD_REQUEST_CAMERA_SETTINGS:
                # self.send_ack(msg)
                self.camera_settings_send()
                return True

            elif msg.command == mavlink.MAV_CMD_REQUEST_STORAGE_INFORMATION:
                self.storage_information_send()
                return True

            elif msg.command == mavlink.MAV_CMD_STORAGE_FORMAT:
                self.storage_format(msg)
                return True

            elif msg.command == mavlink.MAV_CMD_SET_CAMERA_ZOOM:
                # self.log.info(f"***** Zoom {msg}")
                # print(f"Zoom {msg.param2 = }")
                self.set_camera_zoom(msg)
                return True

            elif msg.command == mavlink.MAV_CMD_IMAGE_START_CAPTURE:
                self.image_start_capture(msg)
                return True # return True to indicate that the message has been handled

            elif msg.command == mavlink.MAV_CMD_IMAGE_STOP_CAPTURE:
                self.image_stop_capture(msg)
                return True

            elif msg.command == mavlink.MAV_CMD_VIDEO_START_CAPTURE:
                self.video_start_capture(msg)
                return True

            elif msg.command == mavlink.MAV_CMD_VIDEO_STOP_CAPTURE:
                self.video_stop_capture(msg)
                return True

            elif msg.command == mavlink.MAV_CMD_SET_CAMERA_MODE:
                self.set_camera_mode(msg)
                return True # return True to indicate that the message has been handled

            
        else:
            self.log.debug(f"Unknown command {msg.get_type()} received from {msg.get_srcSystem()}/{msg.get_srcComponent()}")
            return False # return False to indicate that the message has not been handled

        
    def camera_capture_status_send(self):
        """ Information about the status of a capture. Can be requested with a
            MAV_CMD_REQUEST_MESSAGE command."""
        cam = self.camera
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS
        self.mav.camera_capture_status_send(time_since_boot_ms(), # time_boot_ms
                                            0, # image status
                                            0, # video status
                                            0, # image interval
                                            0, # recording time ms
                                            0, # available capacity
                                            0, # image count
                                            )

    def camera_information_send(self):
        """ Information about a camera. Can be requested with a
            MAV_CMD_REQUEST_MESSAGE command."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
        self.camera.camera_information_send()

    def camera_settings_send(self):
        """ Settings of a camera. Can be requested with a
            MAV_CMD_REQUEST_MESSAGE command."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS
        nan = float("nan")
        self.mav.camera_settings_send(time_since_boot_ms(), # time_boot_ms
                                      0, # https://mavlink.io/en/messages/common.html#CAMERA_MODE
                                      NAN, # Current zoom level as a percentage of the full range (0.0 to 100.0, NaN if not known)
                                      NAN, # Current focus level as a percentage of the full range (0.0 to 100.0, NaN if not known)
                                      )

    def storage_information_send(self):
        """ Information about a storage medium. This message is sent in response to
            MAV_CMD_REQUEST_MESSAGE."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION
        self.camera.storage_information_send()
        # self.mav.storage_information_send(time_since_boot_ms(), # time_boot_ms
        #                                   0, # storage_id
        #                                   0, # storage_count
        #                                   0, # status
        #                                   0, # total_capacity
        #                                   0, # used_capacity
        #                                   0, # available_capacity
        #                                   0, # read_speed
        #                                   0, # write_speed
        #                                   0, # storage_type
        #                                   0, # total_capacity
        #                                   0, # used_capacity
        #
        #                                   )

    def storage_format(self, msg):
        """ A message containing the result of the format attempt (asynchronous)."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT
        self.log.error(f"storage_format_send not implemented")
        # do format storage
        storage_id = msg.param1
        format =    msg.param2
        reset_image_log = msg.param3
        
        self.mav.storage_information_send(  time_since_boot_ms(), # time_boot_ms
                                            storage_id, # storage_id
                                            0, # storage_count
                                            0, # status
                                            0, # total_capacity
                                            0, # used_capacity
                                            0, # available_capacity
                                            0, # read_speed
                                            0, # write_speed
                                            0, # storage_type
                                            0, # total_capacity
                                            0, # used_capacity
                                            0, # available_capacity
                                            0, # read_speed
                                            0, # write_speed
                                            0, # storage_type
                                            0, # total_capacity
                                            0, # used_capacity
                                            0, # available_capacity
                                            0, # read_speed
                                            0, # write_speed
                                            0, # storage_type
                                            0, # total_capacity
                                            0, # used_capacity
                                            0, # available_capacity
                                            0, # read_speed
                                            0, # write_speed
                                            0, # storage_type
                                            0, # total_capacity
                                            0, # used_capacity
                                            0, # available_capacity
                                            0, # read_speed
                                            0, # write_speed
                                            0, # storage_type
                                            )



    def set_camera_zoom(self, msg):
        """ Set the camera zoom """
        # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
        self.log.error(f"set_camera_zoom not implemented")
        pass
        
        
    def image_start_capture(self, msg):
        """Start image capture sequence."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
        self.camera.image_start_capture(msg.param2, msg.param3)
        # self.log.error(f"image_start_capture not implemented")
        # start capture thread. each capture send CAMERA_IMAGE_CAPTURED

        # self.mav.camera_image_captured_send(time_since_boot_ms(), # time_boot_ms
        #                                               0, # time_utc
        #                                               0, # camera_id
        #                                               0, # lat
        #                                               0, # lon
        #                                               0, # alt
        #                                               0, # relative_alt
        #                                               0, # q
        #                                               0, # image_index
        #                                               0, # capture_result
        #                                               0, # file_url
        #                                               )
        # mavlink.MAVLink().camera_image_captured_send(time_since_boot_ms(), # time_boot_ms
        #                                              0, # time_utc
        #                                              0, # camera_id
        #                                              0, # lat
        #                                              0, # lon
        #                                              0, # alt
        #                                              0, # relative_alt
        #                                              0, # q
        #                                              0, # image_index
        #                                              0, # capture_result
        #                                              0, # file_url
        #                                              )

        # self.mav.camera_image_captured_send(


    def image_stop_capture(self, msg):
        """Stop image capture sequence"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE
        self.log.error(f"image_stop_capture not implemented")

    def video_start_capture(self, msg):
        """Start video capture"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE
        self.log.error(f"video_start_capture not implemented")

    def video_stop_capture(self, msg):
        """Stop video capture"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE
        self.log.error(f"video_stop_capture not implemented")

    def set_camera_mode(self, msg):
        """ Set the camera mode"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE
        self.log.error(f"set_camera_mode not implemented")

    def start_capture(self, msg):
        """Start capture"""
        self.log.error(f"start_capture not implemented")



    def close(self):
        """Close the connection to the camera"""
        super().close()
        self.log.debug(f"Closed connection to camera")
        return True
    



IndentationError: unexpected indent (2720933931.py, line 15)

In [ ]:
show_doc(CameraServer)

In [ ]:
MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA
# cli = GimbalClient(mav_connection=None, source_component=11, mav_type=MAV_TYPE_GCS, debug=False)
# gim1 = GimbalServer(mav_connection=None, source_component=22, mav_type=MAV_TYPE_CAMERA, debug=False)

con1, con2 = "udpin:localhost:14445", "udpout:localhost:14445"
# con1, con2 = "/dev/ttyACM0", "/dev/ttyUSB0"
with MAVCom(con1, source_system=111, debug=False) as client:
    with MAVCom(con2, source_system=222, debug=False) as server:
        cam:CameraClient = client.add_component(CameraClient(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
        server.add_component(CameraServer(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
        
        cam.wait_heartbeat(target_system=222, target_component=22, timeout=1)
        time.sleep(0.1)
        cam.set_target(222, 22)

        # client.component[11]._test_command(222, 22, 1)
        # for i in range (1)  :
        #     time.sleep(0.01)
        cam.request_camera_capture_status()
        

In [ ]:
# | hide
assert False, "Stop here"

In [ ]:
show_doc(CameraClient)

#### Test locally using UDP ports

#### Starting a client and server
 > on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`

In [ ]:
from UAV.mavlink.mavcom import MAVCom
from UAV.mavlink.component import Component, mavutil
import time

MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA

class Cam1(Component):
    def __init__(self,mav_connection, source_component, mav_type, debug=False):
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)

class Cam2(Component):
    def __init__(self,mav_connection, source_component, mav_type, debug=False):
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)
class Cli(Component):
    def __init__(self,mav_connection, source_component, mav_type, debug=False):
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)

In [ ]:
# Test sending a command and receiving an ack from client to server
with MAVCom("udpin:localhost:14445", source_system=111, debug=False) as client:
    with MAVCom("udpout:localhost:14445", source_system=222, debug=False) as server:
        client.add_component(Cli(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 23, debug=False))
        
        for key, comp in client.component.items():
            if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                print ("*** Received heartbeat **** " )
        NUM_TO_SEND = 2
        for i in range(NUM_TO_SEND):
            client.component[11]._test_command(222, 22, 1)
            client.component[11]._test_command(222, 23, 1)
            
        client.component[11]._test_command(222, 24, 1)

    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    print()
    print(f"{client.source_system = } \n{client.summary()} \n")
    print(f"{server.source_system = } \n{server.summary()} \n")

    assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
    assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
    assert client.component[11].num_acks_drop == 1
    assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
    assert server.component[23].num_cmds_rcvd == NUM_TO_SEND

In [ ]:
#| Hide
# assert False, "Stop here"

In [ ]:
show_doc(Component.set_source_compenent)

In [ ]:
show_doc(Component.send_heartbeat)

In [ ]:
show_doc(Component.on_command_rcvd)

In [ ]:
show_doc(Component.send_command)

In [ ]:
show_doc(Component.wait_heartbeat)

In [ ]:
show_doc(Component.wait_ack)

In [ ]:
show_doc(Component.send_ping)

#### Test with Serial ports
Test using a Pixhawk connected via telemetry 2 and USB serial ports.
CamClient is set to udpin:localhost:14445 and CamServer is set to udpout:localhost:14435 udpin is so that the client can receive UDP from the mavproxy server at localhost:14445
mavproxy.py --master=/dev/ttyACM1 --baudrate 57600 --out udpout:localhost:14445 mavproxy.py --master=/dev/ttyACM3 --baudrate 57600 --out udpout:localhost:14435

In [ ]:
# Test sending a command and receiving an ack from client to server
with MAVCom("/dev/ttyACM0", source_system=111, debug=False) as client:
    with MAVCom("/dev/ttyUSB0", source_system=222, debug=False) as server:
        client.add_component(Cli(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 23, debug=False))
        
        for key, comp in client.component.items():
            if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                print ("*** Received heartbeat **** " )
        NUM_TO_SEND = 2
        for i in range(NUM_TO_SEND):
            client.component[11]._test_command(222, 22, 1)
            client.component[11]._test_command(222, 23, 1)
            
        client.component[11]._test_command(222, 24, 1)

    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    print()
    print(f"{client.source_system = } \n{client.summary()} \n")
    print(f"{server.source_system = } \n{server.summary()} \n")

    assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
    assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
    assert client.component[11].num_acks_drop == 1
    assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
    assert server.component[23].num_cmds_rcvd == NUM_TO_SEND

> For debugging help see http://localhost:3000/tutorials/mavlink_doc&debug.html and http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()